# DATA PROCESSING AND CALCULATION OF xG-ADJUSTED FPL POINTS

## Setup and pre-processing

In [1]:
# give the number of the latest FPL round
latest_gameweek = 18

In [2]:
# import basic libraries
import pandas as pd
import numpy as np
import json
import requests
from scipy.stats import poisson

# allow more data columns to be shown than usually
pd.set_option('max_columns',100)

In [3]:
# import player data 
filepath = '../data/fbref/player_stats_week' + str(latest_gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)#, encoding='latin-1')
playerStats.head()

,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK,G+A-PK,xG,npxG,xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA,Matches
Rk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Patrick van Aanholt\Patrick-van-Aanholt,nl NED,DF,Crystal Palace,28,1990,16,16,1379,2,0,1,1,0,0,0.13,0.00,0.13,0.07,0.07,1.5,0.7,0.9,0.09,0.06,0.15,0.05,0.10,Matches
2,Max Aarons\Max-Aarons,eng ENG,MF,Norwich City,19,2000,16,16,1440,0,0,0,0,4,0,0.00,0.00,0.00,0.00,0.00,0.3,0.3,2.5,0.02,0.16,0.18,0.02,0.18,Matches
3,Tammy Abraham\Tammy-Abraham,eng ENG,FW,Chelsea,21,1997,17,16,1313,11,3,0,0,2,0,0.75,0.21,0.96,0.75,0.96,8.4,8.4,2.1,0.58,0.14,0.72,0.58,0.72,Matches
4,Che Adams\Che-Adams,eng ENG,FW,Southampton,23,1996,13,6,491,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,1.4,1.4,0.2,0.26,0.04,0.30,0.26,0.30,Matches
5,Adrián\Adrian,es ESP,GK,Liverpool,32,1987,10,8,783,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches


In [4]:
# import team data and pre-process
filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col=0, encoding='latin-1')

# change team names to match convention used in the FPL data
teamStats.loc[teamStats['Squad']=='Brighton & Hove Albion','Squad'] = 'Brighton'
teamStats.loc[teamStats['Squad']=='Manchester United','Squad'] = 'Manchester Utd'
teamStats.loc[teamStats['Squad']=='Newcastle United','Squad'] = 'Newcastle Utd'
teamStats.loc[teamStats['Squad']=='Sheffield United','Squad'] = 'Sheffield Utd'
teamStats.loc[teamStats['Squad']=='West Ham United','Squad'] = 'West Ham'
teamStats.loc[teamStats['Squad']=='Tottenham Hotspur','Squad'] = 'Tottenham'
teamStats.loc[teamStats['Squad']=='Wolverhampton Wanderers','Squad'] = 'Wolves'

teamStats.head()

,Squad,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
Rk,,,,,,,,,,,,,,,,,,
1,Liverpool,17,16,1,0,42,14,28,49,31.4,16.8,14.7,0.86,W W W W W,53073,Mohamed Salah Sadio ManÃ© - 9,AdriÃ¡n,NaN
2,Leicester City,18,12,3,3,41,14,27,39,30.2,20.1,10.2,0.57,W W W D L,32027,Jamie Vardy - 17,Kasper Schmeichel,NaN
3,Manchester City,18,12,2,4,50,20,30,38,45.7,17.2,28.5,1.58,D W L W W,54370,Raheem Sterling Sergio AgÃ¼ero - 9,Ederson,NaN
4,Chelsea,18,10,2,6,33,25,8,32,31.4,18.6,12.8,0.71,L W L L W,40557,Tammy Abraham - 11,Kepa Arrizabalaga,NaN
5,Sheffield Utd,18,7,7,4,22,16,6,28,21.2,19.5,1.7,0.10,D L W W W,30863,Lys Mousset - 5,Dean Henderson,NaN


In [5]:
# fetch FPL data online
data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
df = pd.DataFrame(data['elements'])
df.set_index('id',inplace=True)

# fetch data locally
#df = pd.read_csv('../data/data_week' + str(latest_gameweek) + '.csv', index_col=0)#,encoding='latin-1')

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,in_dreamteam,influence,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_saved,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,0,24,100.0,100.0,0,69140,0,0,-3,3,0.8,0,2,1.2,1.2,0,Shkodran,0.7,2,0,7.5,False,19.8,90,,2019-11-28T23:00:21.541666Z,52,0,0,0,69140.jpg,4.0,0,0,Mustafi,0.3,False,None,a,1,3,54.0,4,7947,42,32342,135,0.1,0.8,Mustafi,0
2,0,0,26,75.0,75.0,0,98745,0,0,-1,1,22.7,0,2,0.5,0.5,0,Héctor,0.2,6,0,6.9,False,30.8,262,Hamstring injury - 75% chance of playing,2019-12-09T20:00:21.228098Z,54,0,0,0,98745.jpg,0.3,0,0,Bellerín,0.3,False,None,d,1,3,15.0,1,34674,24,34578,341,0.0,0.2,Bellerín,2
3,2,1,196,0.0,0.0,1,111457,0,0,-3,3,149.6,0,2,0.0,0.0,0,Sead,1.3,17,0,37.8,False,174.4,928,Ankle injury - Expected back 11 Jan,2019-12-15T19:30:19.136195Z,52,0,0,0,111457.jpg,1.9,0,0,Kolasinac,0.6,False,None,i,1,3,55.0,27,46958,11,112204,1076,0.2,5.2,Kolasinac,2
4,2,3,150,100.0,100.0,2,154043,0,0,-4,4,143.5,1,2,2.0,2.0,6,Ainsley,1.5,14,0,35.4,False,182.8,760,,2019-09-22T18:00:10.824841Z,46,0,0,0,154043.jpg,2.9,1,0,Maitland-Niles,2.6,False,None,a,1,3,33.0,26,548323,1040,569302,593,0.3,5.7,Maitland-Niles,0
5,0,2,224,100.0,0.0,2,39476,0,0,-1,1,30.7,0,2,0.8,0.0,0,Sokratis,0.3,23,1,44.3,False,340.2,1350,,2019-12-15T19:30:19.149412Z,49,0,0,0,39476.jpg,2.2,0,0,Papastathopoulos,1.4,False,None,a,1,3,71.0,33,107545,179,148456,862,0.1,6.7,Sokratis,5


## Probability to keep a clean sheet

Here, we estimate for each team the probability that the team keeps a clean sheet (against average opposition). We do this by first calculating the expected goals allowed per game for each team. Then, we assume that conceding goals follows a Poisson distribution, from which we then get the desired probability.

In [6]:
teamStats['xGA per game'] = teamStats['xGA'] / teamStats['MP']
teamStats['probability no goals allowed'] = poisson.pmf(0,teamStats['xGA per game'])
teamStats

,Squad,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xGA per game,probability no goals allowed
Rk,,,,,,,,,,,,,,,,,,,,
1,Liverpool,17,16,1,0,42,14,28,49,31.4,16.8,14.7,0.86,W W W W W,53073,Mohamed Salah Sadio ManÃ© - 9,AdriÃ¡n,NaN,0.988235,0.372233
2,Leicester City,18,12,3,3,41,14,27,39,30.2,20.1,10.2,0.57,W W W D L,32027,Jamie Vardy - 17,Kasper Schmeichel,NaN,1.116667,0.327369
3,Manchester City,18,12,2,4,50,20,30,38,45.7,17.2,28.5,1.58,D W L W W,54370,Raheem Sterling Sergio AgÃ¼ero - 9,Ederson,NaN,0.955556,0.384598
4,Chelsea,18,10,2,6,33,25,8,32,31.4,18.6,12.8,0.71,L W L L W,40557,Tammy Abraham - 11,Kepa Arrizabalaga,NaN,1.033333,0.355819
5,Sheffield Utd,18,7,7,4,22,16,6,28,21.2,19.5,1.7,0.10,D L W W W,30863,Lys Mousset - 5,Dean Henderson,NaN,1.083333,0.338465
6,Wolves,18,6,9,3,26,22,4,27,22.9,18.1,4.8,0.26,D W D L W,31237,RaÃºl JimÃ©nez - 7,Rui PatrÃ­cio,NaN,1.005556,0.365841
7,Tottenham,18,7,5,6,32,26,6,26,22.3,23.3,-0.9,-0.05,W L W W L,59642,Harry Kane - 9,Paulo Gazzaniga,NaN,1.294444,0.274050
8,Manchester Utd,18,6,7,5,26,22,4,25,28.3,17.6,10.6,0.59,D W W D L,72358,Marcus Rashford - 10,David de Gea,NaN,0.977778,0.376146
9,Newcastle Utd,18,7,4,7,18,24,-6,25,14.5,28.4,-13.9,-0.77,D W W L W,46049,Jonjo Shelvey - 5,Martin DÃºbravka,NaN,1.577778,0.206433


## xG-adjusted points

Next, we determine for each player their 'adjusted points'. To do this, we first subtract for each player all the points they have accumulated through goals, assists and clean sheets. Then, we add points for each player based on their expected goals, assists and clean sheets. This gives a much improved estimate of each player's true point generating capability. 

In [7]:
def incorporate_xG(indicator, ix):
    xG = playerStats.loc[indicator, 'xG'].values[0]
    if df.loc[ix, 'element_type']<=2:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] -  6 * (df.loc[ix, 'goals_scored'] - xG)
    elif df.loc[ix, 'element_type']==3:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] -  5 * (df.loc[ix, 'goals_scored'] - xG)
    elif df.loc[ix, 'element_type']==4:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] -  4 * (df.loc[ix, 'goals_scored'] - xG)

In [8]:
# always run 'team_xGA' AFTER 'incorporate_xG'
def team_xGA(indicator, ix):
    team = team_names[df.loc[ix, 'team']-1]
    clean_sheets = df.loc[ix, 'clean_sheets']
    probability_cleanSheet = teamStats.loc[teamStats['Squad']==team, 'probability no goals allowed'].values[0]
    if df.loc[ix, 'element_type']<=2:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  \
                    4 * (df.loc[ix, 'clean_sheets'] - df.loc[ix, 'games played']*probability_cleanSheet)
    elif df.loc[ix, 'element_type']==3:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  \
                    (df.loc[ix, 'clean_sheets'] - df.loc[ix, 'games played']*probability_cleanSheet)

A player who gives an assist that directly leads to a shot, is assigned the xG-value of the shot in xA (expected assists), i.e. xA is a measure of 'goal assists'. In FPL, however, the definition of an assist is somewhat more relaxed, e.g. goals resulting from a rebound of parried shot will award an assist to the player making the initial shot. For this reason, we calculate the total number of assists awarded in FPL and the total sum of xA of all players and get an estimate of the proportion of assists that xA covers in FPL. Then, we modify that proportion of players assists based on their xA.

In [9]:
# always run 'xA' AFTER 'incorporate_xG'
def xA(indicator, ix):
    xA = playerStats.loc[indicator, 'xA'].values[0]
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  3 * (xA_proportion*df.loc[ix, 'assists'] - xA)

In [10]:
team_names = np.sort(playerStats['Squad'].unique())
xA_proportion = playerStats['xA'].sum()/df['assists'].sum()
df['points_per_game'] = df['points_per_game'].astype(float)
df['games played'] = df['total_points']/df['points_per_game']
#df['games played'] = df['minutes']/90.0
xA_proportion

0.7490867579908677

## Main loop for assigning adjusted points

Below is the main loop where we calculate adjusted points for each player. Calculation of the adjusted points itself is straightforward, but there is some work required to match players in two different data sets. Comparing player names in both data sets gives unique matches in many cases, but some special cases need to be covered through individual solutions.

In [11]:
for ix in df[df['minutes']>0].index:    
    name = df.loc[ix, 'web_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u').replace('ö', 'o').\
                        replace('ä', 'a')
    indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains(name)
    if playerStats.loc[indicator].shape[0]==1:
        incorporate_xG(indicator, ix)
        team_xGA(indicator, ix)
        xA(indicator, ix)
    elif playerStats.loc[indicator].shape[0]==0:
        first_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u')\
                                                                    .replace('ö', 'o').replace('ä', 'a')
        first_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(first_name)
        if playerStats.loc[first_name_indicator].shape[0]==1:
            incorporate_xG(first_name_indicator, ix)
            team_xGA(first_name_indicator, ix)
            xA(first_name_indicator, ix)
        else:
            names = ['rodrigo','garcia','chicharito']
            names_playerStats_index = [76, 380, 335]
            if name in names:
                name_ix = names.index(name)
                exceptional_case_indicator = playerStats.index == names_playerStats_index[name_ix]
                incorporate_xG(exceptional_case_indicator, ix)
                team_xGA(exceptional_case_indicator, ix)
                xA(exceptional_case_indicator, ix)
            else:
                print(str(ix) + ': no player found.')
    elif playerStats.loc[indicator].shape[0]>1:
        full_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '') \
            + df.loc[ix, 'second_name'].lower().replace(' ', '').replace('-', '')
        full_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(full_name)
        if playerStats.loc[full_name_indicator].shape[0]==1:
            incorporate_xG(full_name_indicator, ix)
            team_xGA(full_name_indicator, ix)
            xA(full_name_indicator, ix)
        else:
            team = team_names[df.loc[ix, 'team']-1]
            team_indicator = playerStats['Squad']==team
            if playerStats.loc[indicator & team_indicator].shape[0]==1:
                incorporate_xG(indicator & team_indicator, ix)
                team_xGA(indicator & team_indicator, ix)
                xA(indicator & team_indicator, ix)
            else:
                if name=='son':
                    exceptional_case_indicator = playerStats.index == 176
                    incorporate_xG(exceptional_case_indicator, ix)
                    team_xGA(exceptional_case_indicator, ix)
                    xA(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': non-unique name.')
    else:
        print(str(ix) + 'Player not found')

In [12]:
df['adjusted points per game'] = df['adjusted points'] / df['games played']

In [13]:
# give a sorted list showing the players with highest 'adjusted points per game'
df[['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game']]\
                                .sort_values(by='adjusted points per game', ascending=False)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game
id,,,,,,
386,Chicharito,2.000000,9,4.5,25.000000,12.500000
191,Salah,14.054054,104,7.4,93.542342,6.655897
215,De Bruyne,16.901408,120,7.1,110.333132,6.528044
166,Vardy,17.974684,142,7.9,114.863699,6.390304
192,Mané,15.972222,115,7.2,101.014566,6.324390
281,McGovern,2.000000,11,5.5,12.562221,6.281110
214,Sterling,17.142857,96,5.6,101.698595,5.932418
233,Rashford,18.103448,105,5.8,105.863699,5.847709
167,Iheanacho,3.013699,22,7.3,17.605479,5.841818


In [14]:
# save data
filepath = '../data/data_week' + str(latest_gameweek) + str('.csv')
df.to_csv(filepath)

filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats.to_csv(filepath)

Below we check how well the total xG matches the total scored goals.

In [15]:
playerStats['xG'].sum()

485.20000000000005

In [16]:
df['goals_scored'].sum()

489

In [17]:
playerStats['xG'].sum()/df['goals_scored'].sum()

0.9922290388548058